In [7]:
import subprocess

In [ ]:
import subprocess

log_file = "./logs/run_log.txt"

with open(log_file, "w", encoding="utf-8") as f:
    process = subprocess.Popen(
        [
            "python", "./scripts/material_information_with_3ds_fm_gateway.py", 
            "--supplier_name", "Milan Creative Collectibles S.r.l.",
            "--supplier_id", "uuid:6901bbfc-bac6-452c-a1ac-a861655e3150",
            "--usage", "Pilot",
            "--usage_id", "dsmatdata:pilot_usageStatus",
            "--usage_restriction", "dsmatdata:none_usageRestriction",
            "--pdf_path", "./documents/S25255.pdf",
            "--system_prompt_path", "./prompts/material_extraction_system_prompt.txt"
        ],
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        text=True,
    )

    for line in process.stdout:
        print(line, end="")   # Show live output in console
        f.write(line)         # Write to log file

    process.wait()

print("✅ Script finished with code:", process.returncode)

Using parameters:
  Supplier Name: Milan Creative Collectibles S.r.l.
  Supplier ID: uuid:6901bbfc-bac6-452c-a1ac-a861655e3150
  Usage: Pilot
  Usage ID: dsmatdata:pilot_usageStatus
  Usage Restriction: dsmatdata:none_usageRestriction
PDF Path: ./documents/S25255.pdf
System Prompt Path: ./prompts/material_extraction_system_prompt.txt
System prompt loaded.
Extracting text (including OCR) from: ./documents/S25255.pdf
Sending text to FM Gateway for structured material extraction...
Saving output to: ./documents/processed/S25255_processed_with_pdf_extraction_fm_gateway.json
âœ… Extraction complete.
✅ Script finished with code: 0


In [ ]:
# Install required dependencies from requirements.txt
# subprocess.run(["pip", "install", "-r", "requirements.txt"])

In [ ]:
method = "FM Gateway with pdf extraction"  # Options: "OpenAI without pdf extraction", "OpenAI with pdf extraction", "FM Gateway with pdf extraction"
if method == "OpenAI without pdf extraction":
    subprocess.run(["python", "extract_materials_info_openai_no_pdf_extraction.py"])
elif method == "OpenAI with pdf extraction":
    subprocess.run(["python", "extract_materials_info_openai_with_pdf_extraction.py"])
elif method = "FM Gateway with pdf extraction":
    subprocess.run(["python", "extract_materials_info_fm_gateway_with_pdf_extraction.py"])

In [10]:
subprocess.run(["python", "extract_materials_info_fm_gateway_with_pdf_extraction.py"], 
               args = ["--supplier_name", "Milan Creative Collectibles S.r.l.",
                       "--supplier_id", "uuid:6901bbfc-bac6-452c-a1ac-a861655e3150",
                       "--usage", "Pilot",
                       "--usage_id", "dsmatdata:pilot_usageStatus",
                       "--usage_restriction", "dsmatdata:none_usageRestriction",
                       "--pdf_path", "./documents/S25255.pdf",
                       "--system_prompt_path", "./prompts/material_extraction_system_prompt.txt"], 
                       )

TypeError: Popen.__init__() got multiple values for argument 'args'